In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature


import trax_util as trx

import importlib

import time as TimeUtils

import sys
# import modules in other directories
sys.path.append('/glade/work/juliob/PyRegridding/Regridder/')


import esmfRegrid as erg


importlib.reload( trx )


In [ ]:
Ret=6378.1

In [ ]:
def write_netcdf(time,time_bnds,lat,lon,data,fname):
    dims   = ["nlon","nlat","ntime","nbnd"]
    coords = dict( 
        lon  = ( ["lon"],lon ),
        lat  = ( ["lat"],lat ),
        time = ( ["time"], time ), 
            )
    dS = xr.Dataset( coords=coords  )
    dS["time_bnds"]=time_bnds
    Dar = xr.DataArray( data=data, dims=('time','lat','lon',),
                        attrs=dict( description='TC-precipitation',units='ms-1',) ,) 
    dS["TCPRECT"]=Dar
    
    dS.to_netcdf( fname )


## create ESMF regridding objects -- takes a while

In [ ]:
scrip_dir = '/glade/p/cesmdata/cseg/inputdata/share/scripgrids/'
dst_scrip = scrip_dir + 'fv0.23x0.31_141008.nc'
src_scrip = scrip_dir + 'ne120np4_pentagons_100310.nc'

dst1d_scrip = scrip_dir + 'fv0.9x1.25_141008.nc'

#ne120 ==> latlon 0.25 degree
regrd, srcf, dstf = erg.Regrid(srcScrip = src_scrip , 
                                srcType  = 'mesh'  ,
                                dstScrip = dst_scrip ,
                                dstType  = 'grid'   )


In [ ]:
DstFieldData = dstf.data
nx,ny = np.shape( DstFieldData ) # Note ESMF objects are transposed 
print(nx,ny)

In [ ]:
Scr=xr.open_dataset( dst_scrip )
latQd_yx = Scr.grid_center_lat.values.reshape( (ny, nx ) )
lonQd_yx = Scr.grid_center_lon.values.reshape( (ny, nx ) )
lonQd = lonQd_yx[0,:]
latQd = latQd_yx[:,0]

## Read in a track file

In [ ]:
power_wind=(1./6.)**(0.11)
trk=trx.readtrx( trx.rcp85fname(sst='sst2') , power_wind=power_wind )

In [ ]:
nstorms,ntraxtime = np.shape( trk.lat )
print( trk.hour[0,0:2])
prectrk=np.zeros((nstorms,ntraxtime) )

## Reshape track stuff 

In [ ]:
lonR=trk.lon.reshape( nstorms*ntraxtime )
latR=trk.lat.reshape( nstorms*ntraxtime )
yearR=trk.year.reshape( nstorms*ntraxtime )
monthR=trk.month.reshape( nstorms*ntraxtime )
dayR=trk.day.reshape( nstorms*ntraxtime )
hourR=trk.hour.reshape( nstorms*ntraxtime )

prectrkR = np.zeros( (nstorms*ntraxtime ))

In [ ]:
"""
drc='/glade/campaign/cgd/ccr/nanr/HPSS/tokeep/TIMESLICE/atm/proc/tseries/hourly3/PRECT/'
basename = 'f.e13.FAMIPC5.ne120_ne120.RCP85_2070_2099_sst2.001.cam.h4.PRECT.'
y=2070
yA=str(y).zfill(4)
fname=drc+basename+yA+'010100Z-'+yA+'123121Z.nc'
dQ=xr.open_dataset( fname )
dQ
"""

In [ ]:
#print(list(dQ.variables))
yearm=trk.year+trk.month/12.
oop=np.where( (yearm[:,0]>2069.9) & (yearm[:,0]<2071.) )
#print(np.min(oop[0]),np.max(oop[0]) )
#print( yearm[ oop[0],0] )
loo=np.linspace(1.,501,num=5)
tic_overall = TimeUtils.perf_counter()

drc='/glade/campaign/cgd/ccr/nanr/HPSS/tokeep/TIMESLICE/atm/proc/tseries/hourly3/PRECT/'
basename = 'f.e13.FAMIPC5.ne120_ne120.RCP85_2070_2099_sst2.001.cam.h4.PRECT.'
drc_o='/glade/campaign/cgd/amp/juliob/TC-cesm1/precip/'
basename_o = 'f.e13.FAMIPC5.ne120_ne120.RCP85_2070_2099_sst2.001.cam.h4.TCPRECT.'

for y in np.arange(start=2071,stop=2073):
    yA=str(y).zfill(4)
    fname=drc+basename+yA+'010100Z-'+yA+'123121Z.nc'
    fname_o=drc_o+basename_o+yA+'010100Z-'+yA+'123121Z.nc'
    dQ=xr.open_dataset( fname )
    nt = dQ.dims['time']
    prec_TC = np.zeros( (nt , ny, nx ) )
    
    for t in np.arange(nt):
        #for t in np.arange(110):
        time=dQ.time[t].values.item()

        prcQd = dQ.PRECT[t,:].values

        prcQd_yx = erg.HorzRG(aSrc = prcQd , 
                    regrd = regrd , 
                    srcField=srcf , 
                    dstField=dstf , 
                    srcGridkey='c' ,
                    dstGridkey='yx' ) 

        oop=np.where((yearR==time.year) &
                     (monthR==time.month) &
                     (dayR==time.day) &
                     (hourR==time.hour) )
    
        for u in oop[0]:
            if((yearR[u] == time.year) and
                (monthR[u] == time.month) and 
                (dayR[u] == time.day) and
                (hourR[u] == time.hour) ):
                precTC,js,jn,iw,ie = trx.Prec500grid( lonR[u],latR[u],lonQd,latQd,prcQd_yx)
                #print( np.shape(precTC),np.shape(prec_TC[t,js:jn,iw:ie]),js,jn,iw,ie )
                prec_TC[t,js:jn,iw:ie] = prec_TC[t,js:jn,iw:ie] + precTC
            
                
        if ((t % 100)==0):
            print( nt, t, time.year,time.month,time.day,time.hour )
            
    write_netcdf(dQ.time.data,dQ.time_bnds,latQd,lonQd,prec_TC,fname_o)
    print( "wrote",fname_o )
    
toc_overall = TimeUtils.perf_counter()
pTime = f"Overall time creating prec-500   {toc_overall - tic_overall:0.4f} seconds"
print(pTime)

In [ ]:
dU=xr.open_dataset( fname_o )
print(list(dU.variables))

In [ ]:
poo=np.zeros((ny,nx)) 
for t in np.arange( nt ):
    poo=poo+dU.TCPRECT[t,:,:].values
    if ((t % 100)==0):
        print( nt, t )


In [ ]:
print(np.shape(trk.lon))
#plt.contourf( lonQd,latQd, 1000.*86400.*poo/nt,levels=np.linspace(0.1,2.1,num=11) )
#plt.colorbar()

In [ ]:
TCprecav=1000.*86400.*poo/nt

preclv=np.linspace(0.1,2.1,num=11) 
projPC = ccrs.Robinson(central_longitude=0.)
data_crs = ccrs.PlateCarree()
fig=plt.figure(figsize=(20,12))

d01=prcQd_yx*86400.*1000.
ax1 = fig.add_subplot(1, 1, 1 , projection=projPC )

ax1.coastlines(resolution='110m',color='black',linewidth=4)
ax1.contourf( lonQd, latQd, TCprecav ,levels=preclv,cmap='Blues',transform=data_crs )
for ns in np.arange( 1784 ):
    if (trk.year[ns,0]==2072):
        zoo=ax1.plot( trk.lon[ns,:], trk.lat[ns,:], transform=data_crs  )


ax1.spines['geo'].set_linewidth(4)  # change as needed

In [ ]:
#### STOP HERE???

In [ ]:
prectrk = np.reshape( prectrkR, (nstorms,ntraxtime))
print(np.shape(prectrk))

In [ ]:
plt.scatter( trk.lon.flatten()[0:10_000] ,trk.lat.flatten()[0:10_000], c=1000.*86400*prectrk.flatten()[0:10_000],vmin=5,vmax=100)

In [ ]:
plt.plot(prectrkR)

In [ ]:
plt.plot(np.linspace(0,1))

In [ ]:
 ,trk.lat.flatten()[0:20_000], c=1000.*86400*prectrk.flatten()[0:20_000],vmin=5,vmax=100)

In [ ]:
plt.plot((1000./(21*21))*86400*prectrk[0:66,:].flatten())

In [ ]:
yearm=trk.year+trk.month/12.
oop=np.where( (yearm[:,0]>2098.9) & (yearm[:,0]<2100.) )
print(np.min(oop[0]),np.max(oop[0]) )
print( yearm[ oop[0],0] )

In [ ]:
lonQd=dsQd.lon.values
latQd=dsQd.lat.values
prcQd = prcQd / Nyr

In [ ]:
prcQd_yx = erg.HorzRG(aSrc = prcQd , 
                regrd = regrd , 
                srcField=srcf , 
                dstField=dstf , 
                srcGridkey='c' ,
                dstGridkey='yx' ) 


In [ ]:
prc1d_yx = erg.HorzRG(aSrc = prcQd , 
                regrd = regrd1d , 
                srcField=srcf1d , 
                dstField=dstf1d , 
                srcGridkey='c' ,
                dstGridkey='yx' ) 


In [ ]:
print(np.shape(prcQd_yx))
Scr=xr.open_dataset( dst_scrip )
latQd_yx = Scr.grid_center_lat.values.reshape( (768, 1152 ) )
lonQd_yx = Scr.grid_center_lon.values.reshape( (768, 1152 ) )
#plt.contour(latQd_yx)
#plt.contour(lonQd_yx)


In [ ]:
print(np.shape(prc1d_yx))
ny1d,nx1d = np.shape(prc1d_yx)
Scr=xr.open_dataset( dst1d_scrip )
lat1d_yx = Scr.grid_center_lat.values.reshape( (ny1d, nx1d ) )
lon1d_yx = Scr.grid_center_lon.values.reshape( (ny1d, nx1d ) )
#plt.contour(latQd_yx)
#plt.contour(lonQd_yx)


In [ ]:
preclv=np.linspace(0,32,num=16+1)
projPC = ccrs.Robinson(central_longitude=0.)
data_crs = ccrs.PlateCarree()
fig=plt.figure(figsize=(20,12))

d01=prcQd_yx*86400.*1000.
ax1 = fig.add_subplot(1, 1, 1 , projection=projPC )
ax1.coastlines(resolution='110m',color='black',linewidth=4)
ax1.contourf( lonQd_yx, latQd_yx, d01 ,levels=preclv,cmap='Blues',transform=data_crs )
ax1.spines['geo'].set_linewidth(4)  # change as needed


In [ ]:
preclv=np.linspace(0,24,num=16+1)
projPC = ccrs.Robinson(central_longitude=180.)
data_crs = ccrs.PlateCarree()
fig=plt.figure(figsize=(20,12))

d01=prc1d_yx*86400.*1000.
ax1 = fig.add_subplot(1, 1, 1 , projection=projPC )
ax1.coastlines(resolution='110m',color='black',linewidth=4)
po=ax1.contourf( lon1d_yx, lat1d_yx, d01 ,levels=preclv,cmap='Blues',transform=data_crs )
ax1.spines['geo'].set_linewidth(4)  # change as needed

cax = plt.axes([0.93, 0.25, 0.01, 0.5])
cbar=plt.colorbar(po,cax=cax,orientation='vertical')
cbar.ax.tick_params(labelsize=20)
ax1.set_title('CESM1 ne120 Jul-Aug-Sep climo',fontsize=32)

In [ ]:
## Should be done 
## np.savez('arrays.npz', arr1=arr1, arr2=arr2, arr3=arr3)
npSaveFile = '/glade/work/juliob/NumPySaves/ne120-JAS-e01-1980-2012.npz'
np.savez( npSaveFile , prec=prcQd, lat=latQd, lon=lonQd )